In [38]:
import pandas as pd
import time
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
import os

from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import matplotlib.pyplot as plt #to display our wordcloud
from PIL import Image #to load our image

In [39]:
for dirname, _, filenames in os.walk('../../data'):
    for filename in filenames:
        print(os.path.join(dirname, filename).replace('\\','/'))

../../data/clean-twitter_en.csv
../../data/clean_public.csv
../../data/labeled_tweets.csv
../../data/public_data_labeled.csv
../../data/twitter_en.csv


In [40]:
# read the data
df = pd.read_csv('../../data/public_data_labeled.csv')

In [41]:
# exclude retweets, which is not useful for modeling later
df = df[~df.full_text.str.contains('RT')].copy()

In [42]:
#1.build lemmatize models, and stopwords dictionary
lemmatizer = WordNetLemmatizer()
stopwords = stopwords.words('English')

In [21]:
# the pipeline to process the raw text data
def pipeline(input:str):
    input = input.lower()
    input=re.sub("\\W"," ",input) # remove special chars
    str_input = re.sub(pattern=r'[^a-zA-z]',repl=' ',string=input) #remove words except digit and words
#     str_output = correct_sentence_spelling(sentence = str_input) # it is too time-consuming
    token_list = word_tokenize(str_input)
    token_list = [lemmatizer.lemmatize(i) for i in token_list]
    word_list = [word for word in token_list if word not in stopwords]
    word_str = ' '.join(word_list)
    return word_str
df.full_text = df.full_text.apply(pipeline)

In [28]:
df.label = df.label.apply(lambda x:1 if x=='Offensive' else 0)
df.groupby('label').count()

In [34]:
from sklearn.feature_extraction.text import CountVectorizer
vectorize = CountVectorizer(analyzer='word',min_df=0.1)
content  = df.full_text
vector = vectorize.fit_transform(content)
data = vector.toarray()
index_name = vectorize.get_feature_names_out()

In [37]:
df.to_csv('../../data/clean_twitter_en.csv')